# Record system's status as json

you change just change these paras to record data in files, finally, this scripts will generate readme.txt and zipfile which filename is datentime and contains these json files

In [52]:
import requests
import pycurl
import json
import datetime,os
import os
import zipfile
import sys

conf = {
    "instance_str" : "140.110.141.135:9090",
    "start_str" : "2018-11-13T09:14:00.781Z",
    "end_str"   : "2018-11-13T09:16:00.781Z",
    "step_str"  : "5s",
    "node_ins_name"  : "tyan0.:18080",
    "gpu_ins_name"  : "tyan0.:18081",
}


This is utility function below, don't change if you don't know.

In [53]:

class PromeDBIns:

    def __init__(self, conf):
        self.total_url_query_list ={}
        self.report_body = {}
        self.instance_str = conf["instance_str"] # "140.110.141.135:9090",
        self.start_str = conf["start_str"] #"2018-10-29T20:10:51.781Z"
        self.end_str   = conf["end_str"] # "2018-10-29T20:12:51.781Z", 
        self.step_str  = conf["step_str"] # "15s"
        now = datetime.datetime.now()
        date_now = "{}{:02}{:02}_{:02}{:02}".format(now.year, now.month, now.day, now.hour, now.minute, now.second)
        self.data_dirname = "./"+date_now
#         total_url_query_list = {}
#         report_body = {}
        self.initReport(self.report_body)
        if not os.path.exists(self.data_dirname):
            os.makedirs(self.data_dirname)
            
    def initReport(self,report_body):
        self.report_body["instance"] =self.instance_str
        self.report_body["starttime"]=self.start_str
        self.report_body["endtime"]  =self.end_str
        self.report_body["step"]     =self.step_str
        self.report_body["ok"]       =self.total_url_query_list
        self.report_body["error"]    =[]        
    
    def storeAsJson(self, response,dst_file):
        dst_file_path = os.path.join(self.data_dirname, dst_file)
 
        if response.status_code == 200 : 
            with open(dst_file_path,"w",encoding='utf8') as f :
                # new
                data = json.loads(response.text)
                json.dump(data,f )
                # original
#                 json.dump(response.text,f )
            self.total_url_query_list[dst_file] = response.url
            return True
        else:
            self.report_body["error"].append(response.url)
            return False

    def zipdir(self, path, zipfl):
        zipf = zipfile.ZipFile(zipfl, 'w', zipfile.ZIP_DEFLATED)
        # ziph is zipfile handle
        for root, dirs, files in os.walk(path):
            for file in files:
                zipf.write(os.path.join(root, file))
        zipf.close()
    def queryDB(self, query_str):

        range_url = "http://"+self.instance_str+"/api/v1/query_range"
        #query_str = 'irate(node_disk_written_bytes_total{instance=~"tyan0.:18080", device=~".*nvme.*"}[5m])'
        range_full_url = "{}?query={}&start={}&end={}&step={}".format(
            range_url,query_str , self.start_str , self.end_str , self.step_str )        
        print(range_full_url)
        
        response = requests.get(range_full_url )

        print(response.text[0:250])  
        return response
        
        
    def queryDBSave(self, query_str , filename):

        
        response = self.queryDB(query_str)
        try:
            dst_file = "./"+filename+".json"
            #self.storeAsJson(range_full_url,dst_file)
            self.storeAsJson(response,dst_file)
            return response
        except:
            print("{} error:{}".format( sys.exc_info()[0],sys.exc_info()[1]) )
            return False

In [54]:
pdbins = PromeDBIns(conf)

# A.diskIO

## a.1 write bytes/s : node_disk_written_bytes

In [55]:
#print(pdbins.queryDBSave( "justTest", "testfile"))

#dst_file = "./node_disk_written_bytes.json"
#range_url = "http://"+instance_str+"/api/v1/query_range"
#query_str = 'irate(node_disk_written_bytes_total{instance=~"tyan0.:18080", device=~".*nvme.*"}[5m])'
#range_full_url = "{}?query={}&start={}&end={}&step={}".format(
#    range_url,query_str , start_str , end_str , step_str )
#
#storeAsJson(range_full_url,dst_file)

rs_disk_written_bytes = pdbins.queryDBSave( 
    'irate(node_disk_written_bytes_total{instance=~"'+conf["node_ins_name"]+'", device=~".*nvme.*"}[5m])',
    "node_disk_written_bytes"
)

http://140.110.141.135:9090/api/v1/query_range?query=irate(node_disk_written_bytes_total{instance=~"tyan0.:18080", device=~".*nvme.*"}[5m])&start=2018-11-13T09:14:00.781Z&end=2018-11-13T09:16:00.781Z&step=5s
{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"device":"nvme0n1","instance":"tyan01:18080","job":"node_exporter_tyan01"},"values":[[1542100440.781,"0"],[1542100445.781,"0"],[1542100450.781,"0"],[1542100455.781,"0"],[154210046


## a.2 read bytes/s : node_disk_read_bytes

In [56]:
#dst_file = "./node_disk_read_bytes.json"
#range_url = "http://"+instance_str+"/api/v1/query_range"
#query_str = 'irate(node_disk_read_bytes_total{instance=~"tyan0.:18080", device=~".*nvme.*"}[5m])'
#range_full_url = "{}?query={}&start={}&end={}&step={}".format(
#    range_url,query_str , start_str , end_str , step_str )
#storeAsJson(range_full_url,dst_file)


rs_disk_read_bytes = pdbins.queryDBSave( 
    'irate(node_disk_read_bytes_total{instance=~"'+conf["node_ins_name"]+'", device=~".*nvme.*"}[5m])',
    "node_disk_read_bytes"
)


http://140.110.141.135:9090/api/v1/query_range?query=irate(node_disk_read_bytes_total{instance=~"tyan0.:18080", device=~".*nvme.*"}[5m])&start=2018-11-13T09:14:00.781Z&end=2018-11-13T09:16:00.781Z&step=5s
{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"device":"nvme0n1","instance":"tyan01:18080","job":"node_exporter_tyan01"},"values":[[1542100440.781,"0"],[1542100445.781,"0"],[1542100450.781,"0"],[1542100455.781,"0"],[154210046


## a.3 disk usage : node_disk_io_time_seconds_total

In [57]:
#dst_file = "./node_disk_io_time.json"
#range_url = "http://"+instance_str+"/api/v1/query_range"
#query_str = 'rate(node_disk_io_time_seconds_total{instance=~"tyan0.:18080", device!~"md.*|dm.*"}[5m])*100'
#range_full_url = "{}?query={}&start={}&end={}&step={}".format(
#    range_url,query_str , start_str , end_str , step_str )
#storeAsJson(range_full_url,dst_file)

rs_disk_io_time_seconds = pdbins.queryDBSave( 
    'irate(node_disk_io_time_seconds_total{instance=~"'+conf["node_ins_name"]+'", device=~".*nvme.*"}[5m])',
    "node_disk_io_time_seconds"
)


http://140.110.141.135:9090/api/v1/query_range?query=irate(node_disk_io_time_seconds_total{instance=~"tyan0.:18080", device=~".*nvme.*"}[5m])&start=2018-11-13T09:14:00.781Z&end=2018-11-13T09:16:00.781Z&step=5s
{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"device":"nvme0n1","instance":"tyan01:18080","job":"node_exporter_tyan01"},"values":[[1542100440.781,"0"],[1542100445.781,"0"],[1542100450.781,"0"],[1542100455.781,"0"],[154210046


# B.system

## b.1 cpu loading : node_load1

In [58]:
#dst_file = "./node_load1.json"
#query_str = 'node_load1{instance=~"tyan0.:18080"}'
#range_url = "http://"+instance_str+"/api/v1/query_range"
#range_full_url = "{}?query={}&start={}&end={}&step={}".format(
#    range_url,query_str , start_str , end_str , step_str )
#storeAsJson(range_full_url,dst_file)

rs_node_load1 = pdbins.queryDBSave( 
    'node_load1{instance=~"'+conf["node_ins_name"]+'"}',
    "node_load1"
)

http://140.110.141.135:9090/api/v1/query_range?query=node_load1{instance=~"tyan0.:18080"}&start=2018-11-13T09:14:00.781Z&end=2018-11-13T09:16:00.781Z&step=5s
{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"__name__":"node_load1","instance":"tyan01:18080","job":"node_exporter_tyan01"},"values":[[1542100440.781,"0.18"],[1542100445.781,"0.25"],[1542100450.781,"0.23"],[1542100455.781,"


## b.2 temperature : node_hwmon_temp_celsius

In [59]:
#dst_file = "./node_hwmon_temp_celsius.json"
#query_str = 'node_hwmon_temp_celsius{instance=~"tyan0.:18080"}'
#range_url = "http://"+instance_str+"/api/v1/query_range"
#range_full_url = "{}?query={}&start={}&end={}&step={}".format(
#    range_url,query_str , start_str , end_str , step_str )
#storeAsJson(range_full_url,dst_file)

rs_hwmon_temp_celsius = pdbins.queryDBSave( 
    'node_hwmon_temp_celsius{instance=~"'+conf["node_ins_name"]+'"}',
    "node_hwmon_temp_celsius"
)

http://140.110.141.135:9090/api/v1/query_range?query=node_hwmon_temp_celsius{instance=~"tyan0.:18080"}&start=2018-11-13T09:14:00.781Z&end=2018-11-13T09:16:00.781Z&step=5s
{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"__name__":"node_hwmon_temp_celsius","chip":"platform_coretemp_0","instance":"tyan01:18080","job":"node_exporter_tyan01","sensor":"temp1"},"values":[[1542100440.781,"64"],[1542100


## b.3 RAM free % : node_memory_MemFree_bytes

In [60]:
## not work 
# (1-(sum by (instance)(node_memory_MemFree_bytes{instance=~"tyan0.:18080"}+ \n 
# node_memory_Buffers_bytes{instance=~"tyan0.:18080"}+node_memory_Cached_bytes{instance=~"tyan0.:18080"}))/ \n
# (sum by (instance) (node_memory_MemTotal_bytes{instance=~"tyan0.:18080"})))*100

## so change
#dst_file = "./node_memory_MemFree_bytes.json"
#query_str = 'node_memory_MemFree_bytes{instance=~"tyan0.:18080"}'
#range_url = "http://"+instance_str+"/api/v1/query_range"
#range_full_url = "{}?query={}&start={}&end={}&step={}".format(
#    range_url,query_str , start_str , end_str , step_str ) 
#storeAsJson(range_full_url,dst_file)


rs_memory_MemFree_bytes = pdbins.queryDBSave( 
    'node_memory_MemFree_bytes{instance=~"'+conf["node_ins_name"]+'"}',
    "node_memory_MemFree_bytes"
)




#dst_file = "./node_memory_Buffers_bytes.json"
#query_str = 'node_memory_Buffers_bytes{instance=~"tyan0.:18080"}'
#range_url = "http://"+instance_str+"/api/v1/query_range"
#range_full_url = "{}?query={}&start={}&end={}&step={}".format(
#    range_url,query_str , start_str , end_str , step_str ) 
#storeAsJson(range_full_url,dst_file)

rs_memory_Buffers_bytes = pdbins.queryDBSave( 
    'node_memory_Buffers_bytes{instance=~"'+conf["node_ins_name"]+'"}',
    "node_memory_Buffers_bytes"
)



#dst_file = "./node_memory_Cached_bytes.json"
#query_str = 'node_memory_Cached_bytes{instance=~"tyan0.:18080"}'
#range_url = "http://"+instance_str+"/api/v1/query_range"
#range_full_url = "{}?query={}&start={}&end={}&step={}".format(
#    range_url,query_str , start_str , end_str , step_str ) 
#storeAsJson(range_full_url,dst_file)

rs_memory_Cached_bytes = pdbins.queryDBSave( 
    'node_memory_Cached_bytes{instance=~"'+conf["node_ins_name"]+'"}',
    "node_memory_Cached_bytes"
)



http://140.110.141.135:9090/api/v1/query_range?query=node_memory_MemFree_bytes{instance=~"tyan0.:18080"}&start=2018-11-13T09:14:00.781Z&end=2018-11-13T09:16:00.781Z&step=5s
{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"__name__":"node_memory_MemFree_bytes","instance":"tyan01:18080","job":"node_exporter_tyan01"},"values":[[1542100440.781,"39083044864"],[1542100445.781,"39082541056"],[1542100450.
http://140.110.141.135:9090/api/v1/query_range?query=node_memory_Buffers_bytes{instance=~"tyan0.:18080"}&start=2018-11-13T09:14:00.781Z&end=2018-11-13T09:16:00.781Z&step=5s
{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"__name__":"node_memory_Buffers_bytes","instance":"tyan01:18080","job":"node_exporter_tyan01"},"values":[[1542100440.781,"6438912"],[1542100445.781,"6438912"],[1542100450.781,"643
http://140.110.141.135:9090/api/v1/query_range?query=node_memory_Cached_bytes{instance=~"tyan0.:18080"}&start=2018-11-13T09:14:00.781Z&end=2018-11-13T09

# C. network

## c.1 node_network_transmit_bytes_total

In [61]:
#dst_file = "./node_network_transmit_bytes_total.json"
#query_str = 'irate(node_network_transmit_bytes_total{instance=~"tyan0.:18080", device=~"enp.*"}[5m])'
#range_url = "http://"+instance_str+"/api/v1/query_range"
#range_full_url = "{}?query={}&start={}&end={}&step={}".format(
#    range_url,query_str , start_str , end_str , step_str ) 
#storeAsJson(range_full_url,dst_file)

rs_network_transmit_bytes_total = pdbins.queryDBSave( 
    'irate(node_network_transmit_bytes_total{instance=~"'+conf["node_ins_name"]+'", device=~"enp.*"}[5m])',
    "node_network_transmit_bytes_total"
)


http://140.110.141.135:9090/api/v1/query_range?query=irate(node_network_transmit_bytes_total{instance=~"tyan0.:18080", device=~"enp.*"}[5m])&start=2018-11-13T09:14:00.781Z&end=2018-11-13T09:16:00.781Z&step=5s
{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"device":"enp103s0","instance":"tyan01:18080","job":"node_exporter_tyan01"},"values":[[1542100440.781,"0"],[1542100445.781,"0"],[1542100450.781,"0"],[1542100455.781,"0"],[15421004


## c.2 node_network_receive_bytes_total

In [62]:
#dst_file = "./node_network_receive_bytes_total.json"
#query_str = 'irate(node_network_receive_bytes_total{instance=~"tyan0.:18080", device=~"enp.*"}[5m])'
#range_url = "http://"+instance_str+"/api/v1/query_range"
#range_full_url = "{}?query={}&start={}&end={}&step={}".format(
#    range_url,query_str , start_str , end_str , step_str ) 
#storeAsJson(range_full_url,dst_file)

rs_network_receive_bytes_total = pdbins.queryDBSave( 
    'irate(node_network_receive_bytes_total{instance=~"'+conf["node_ins_name"]+'", device=~"enp.*"}[5m])',
    "node_network_receive_bytes_total"
)

http://140.110.141.135:9090/api/v1/query_range?query=irate(node_network_receive_bytes_total{instance=~"tyan0.:18080", device=~"enp.*"}[5m])&start=2018-11-13T09:14:00.781Z&end=2018-11-13T09:16:00.781Z&step=5s
{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"device":"enp103s0","instance":"tyan01:18080","job":"node_exporter_tyan01"},"values":[[1542100440.781,"0"],[1542100445.781,"0"],[1542100450.781,"0"],[1542100455.781,"0"],[15421004


## c.3 packet loss : node_netstat_Tcp_RetransSegs

In [63]:
#dst_file = "./node_netstat_Tcp_RetransSegs.json"
#query_str = 'irate(node_netstat_Tcp_RetransSegs{instance=~"tyan0.:18080"}[5m])'
#range_url = "http://"+instance_str+"/api/v1/query_range"
#range_full_url = "{}?query={}&start={}&end={}&step={}".format(
#    range_url,query_str , start_str , end_str , step_str ) 
#storeAsJson(range_full_url,dst_file)


rs_netstat_Tcp_RetransSegs = pdbins.queryDBSave( 
    'irate(node_netstat_Tcp_RetransSegs{instance=~"'+conf["node_ins_name"]+'"}[5m])',
    "node_netstat_Tcp_RetransSegs"
)


http://140.110.141.135:9090/api/v1/query_range?query=irate(node_netstat_Tcp_RetransSegs{instance=~"tyan0.:18080"}[5m])&start=2018-11-13T09:14:00.781Z&end=2018-11-13T09:16:00.781Z&step=5s
{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"instance":"tyan01:18080","job":"node_exporter_tyan01"},"values":[[1542100440.781,"0"],[1542100445.781,"0"],[1542100450.781,"0"],[1542100455.781,"0"],[1542100460.781,"0"],[1542100


# D. GPU

## d.1 load

In [64]:
#dst_file = "./nvidia_gpu_duty_cycle.json"
#query_str = 'nvidia_gpu_duty_cycle{instance=~"tyan0.:18081"}'
#range_url = "http://"+instance_str+"/api/v1/query_range"
#range_full_url = "{}?query={}&start={}&end={}&step={}".format(
#    range_url,query_str , start_str , end_str , step_str ) 
#storeAsJson(range_full_url,dst_file)

rs_gpu_duty_cycle = pdbins.queryDBSave( 
    'nvidia_gpu_duty_cycle{instance=~"'+conf["gpu_ins_name"]+'"}',
    "nvidia_gpu_duty_cycle"
)


http://140.110.141.135:9090/api/v1/query_range?query=nvidia_gpu_duty_cycle{instance=~"tyan0.:18081"}&start=2018-11-13T09:14:00.781Z&end=2018-11-13T09:16:00.781Z&step=5s
{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"__name__":"nvidia_gpu_duty_cycle","instance":"tyan01:18081","job":"nvidia_exporter_tyan01","minor_number":"0","name":"Tesla V100-PCIE-16GB","uuid":"GPU-81fc6336-3dc2-5952-de3f-ab


## d.2 temperature : nvidia_gpu_temperature_celsius

In [65]:
#dst_file = "./nvidia_gpu_temperature_celsius.json"
#query_str = 'nvidia_gpu_temperature_celsius{instance=~"tyan0.:18081"}'
#range_url = "http://"+instance_str+"/api/v1/query_range"
#range_full_url = "{}?query={}&start={}&end={}&step={}".format(
#    range_url,query_str , start_str , end_str , step_str ) 
#storeAsJson(range_full_url,dst_file)

rs_gpu_temperature_celsius = pdbins.queryDBSave( 
    'nvidia_gpu_temperature_celsius{instance=~"'+conf["gpu_ins_name"]+'"}',
    "nvidia_gpu_temperature_celsius"
)

http://140.110.141.135:9090/api/v1/query_range?query=nvidia_gpu_temperature_celsius{instance=~"tyan0.:18081"}&start=2018-11-13T09:14:00.781Z&end=2018-11-13T09:16:00.781Z&step=5s
{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"__name__":"nvidia_gpu_temperature_celsius","instance":"tyan01:18081","job":"nvidia_exporter_tyan01","minor_number":"0","name":"Tesla V100-PCIE-16GB","uuid":"GPU-81fc6336-3dc2-595


## d.3 mem used

In [66]:
#dst_file = "./node_netstat_Tcp_RetransSegs.json"
#query_str = 'nvidia_gpu_memory_used_bytes{instance=~"tyan0.:18081"}'
#range_url = "http://"+instance_str+"/api/v1/query_range"
#range_full_url = "{}?query={}&start={}&end={}&step={}".format(
#    range_url,query_str , start_str , end_str , step_str ) 
#storeAsJson(range_full_url,dst_file)

rs_gpu_memory_used_bytes = pdbins.queryDBSave( 
    'nvidia_gpu_memory_used_bytes{instance=~"'+conf["gpu_ins_name"]+'"}',
    "nvidia_gpu_memory_used_bytes"
)

http://140.110.141.135:9090/api/v1/query_range?query=nvidia_gpu_memory_used_bytes{instance=~"tyan0.:18081"}&start=2018-11-13T09:14:00.781Z&end=2018-11-13T09:16:00.781Z&step=5s
{"status":"success","data":{"resultType":"matrix","result":[{"metric":{"__name__":"nvidia_gpu_memory_used_bytes","instance":"tyan01:18081","job":"nvidia_exporter_tyan01","minor_number":"0","name":"Tesla V100-PCIE-16GB","uuid":"GPU-81fc6336-3dc2-5952-


# save report as readme.txt in json format

In [67]:
readme_path = os.path.join(pdbins.data_dirname,"readme.txt")
with open(readme_path, "w") as f:
    ## simple file
    #f.write(str(total_url_query_list))    
    ## json format
    json.dump(pdbins.report_body,f, sort_keys=True, indent=4)

print("[ok]readme is ready: {}".format(readme_path) )   

## in json 
#json.dumps(report_body, sort_keys=True, indent=4)
## simple format
#for fkey, qvalue in total_url_query_list.items():
#    print("{}=>{} \n".format(fkey,qvalue))

[ok]readme is ready: ./20181113_1752/readme.txt


#  zip these json file

In [68]:


pdbins.zipdir(pdbins.data_dirname, pdbins.data_dirname+".zip")
print("[ok]zip is ready: {}".format(pdbins.data_dirname+".zip"))



[ok]zip is ready: ./20181113_1752.zip
